<a href="https://colab.research.google.com/github/newfull5/AI_Poet-KoGPT2/blob/master/2.%20Transfer_Learning!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!git clone https://github.com/SKT-AI/KoGPT2.git

Cloning into 'KoGPT2'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 61 (delta 30), reused 40 (delta 18), pack-reused 0
Unpacking objects: 100% (61/61), done.


In [3]:
!cd KoGPT2 && pip install -r requirements.txt
!cd KoGPT2 && pip install .

     |████████████████████████████████| 256kB 3.4MB/s 
     |████████████████████████████████| 68.7MB 41kB/s 
     |████████████████████████████████| 1.1MB 41.1MB/s 
     |████████████████████████████████| 481kB 28.6MB/s 
     |████████████████████████████████| 3.1MB 8.5MB/s 
     |████████████████████████████████| 890kB 28.3MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp36-cp36m-linux_x86_64.whl size=471063 sha256=0f1b414aa462a9e9b6e5536116f781d78a8680aea492da35a9df44962e047f8b
  Stored in directory: /root/.cache/pip/wheels/af/60/16/1f8a40e68b85bd9bd7960e91830bca5e40cd113f3220b7e231
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=54a6f5f3373d9c71bdc53281adaf2ee5ceb2ecef4c31e3c578609b299be04e98
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built gluonnlp sacremoses
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      

In [0]:
from kogpt2.pytorch_kogpt2 import get_pytorch_kogpt2_model
from gluonnlp.data import SentencepieceTokenizer
from kogpt2.utils import get_tokenizer
from torch.utils.data import DataLoader
import torch

In [5]:
if torch.cuda.device_count():
  print('구글 만세!')
  PU = 'cuda'
else:
  PU = 'cpu'

구글 만세!


In [0]:
#토큰화와 인덱싱을해서 리턴하는 함수

def dataset (file_path):
  data = []
  tokenizer = SentencepieceTokenizer(get_tokenizer())
  f = open(file_path,'r',encoding='utf-8')

  while True:
    file = f.readline()

    if not file:
      break
    line = tokenizer(file)
    indexing_word = [vocab[vocab.bos_token]]+ vocab[line] + [vocab[vocab.eos_token]]
    data.append(indexing_word)

  f.close()

  return data

In [7]:
_, vocab = get_pytorch_kogpt2_model()

#model.to(torch.device(PU)) #모델 연산 유닛 설정
#model.train() #모델 학습모드로 변경


[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [0]:
import subprocess

def get_gpu_memory_map():
	"""Get the current gpu usage.
	Returns
	-------
	usage: dict
		Keys are device ids as integers.
		Values are memory usage as integers in MB.
	"""
	result = subprocess.check_output(
		[
			'nvidia-smi', '--query-gpu=memory.used',
			'--format=csv,nounits,noheader'
		], encoding='utf-8')
	# Convert lines into a dictionary
	gpu_memory = [int(x) for x in result.strip().split('\n')]
	gpu_memory_map = dict(zip(range(len(gpu_memory)), gpu_memory))
	return gpu_memory_map

In [62]:
get_gpu_memory_map()

{0: 11422}

In [0]:
#모델 불러오기 코드 입니다
from kogpt2.model.torch_gpt2 import GPT2Config, GPT2LMHeadModel

save_path = 'drive/My Drive/Colab Notebooks/KoGPT2_checkpoint/'

kogpt2_config = {
		"initializer_range": 0.02,
		"layer_norm_epsilon": 0.000025,
		"n_ctx": 1024,
		"n_embd": 768,
		"n_head": 12,
		"n_layer": 12,
		"n_positions": 1024,
		"vocab_size": 50000
}

checkpoint = torch.load(save_path+'KoGPT2_checkpoint.tar', map_location=PU)

kogpt2model = GPT2LMHeadModel(config=GPT2Config.from_dict(kogpt2_config))

kogpt2model.load_state_dict(checkpoint['model_state_dict'])

kogpt2model.train()

kogpt2model.to(torch.device(PU))

model = kogpt2model

In [0]:
#torch.save(model.state_dict,'model_state_dict.pth') #모델의 가중치 값을 저장하는 코드입니다.
#model.load_state_dict(torch.load(save_path+'KoGPT2_checkpoint.tar')) #모델의 가중치 값을 불러오는 코드입니다.

#torch.save(model, PATH) #모델 전체를 저장하는 코드입니다.
#model = torch.load(PATH) #모델 전체를 불러오는 코드입니다.

In [10]:
file_path = 'drive/My Drive/poem_data/dataset_combined.txt'
dataset = DataLoader(dataset(file_path), batch_size=100, shuffle=True, pin_memory=True)

learning_rate = 0.000025
epochs = 1000
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

using cached model


In [11]:
for epoch in range(checkpoint['epoch'], epochs+1):
  cnt = 0

  for data in dataset:
    optimizer.zero_grad()

    data = torch.stack(data)
    data = data.transpose(1,0)
    data = data.to(PU)

    output = model(data,labels=data)
    loss, logits = output[:2]
    loss.backward()
    optimizer.step()

    if cnt % 20 == 0:
      print("[+] epoch : {}, cnt : {}, loss : {} [+]".format(epoch, cnt+1, str(loss)[7:12]))

    if epoch % 20 == 0 and cnt == 1:
      torch.save({
          'epoch': epoch,
          'cnt': cnt,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'loss': loss,
          }, save_path+'KoGPT2_checkpoint.tar')
      
    cnt += 1

[+] epoch : 744, cnt : 261, loss : 4.914 [+]
[+] epoch : 745, cnt : 1, loss : 4.770 [+]


KeyboardInterrupt: ignored